# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [81]:
# import libraries
import nltk
nltk.download(['stopwords', 'punkt', 'wordnet'])
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import pandas as pd
from sqlalchemy import create_engine
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
#from sklearn.base import BaseEstimator, TransformerMixin
import pickle

[nltk_data] Downloading package stopwords to /Users/sehe/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/sehe/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/sehe/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [82]:
# load data from database
engine = create_engine('sqlite:///disaster_response_pipeline.db')
df = pd.read_sql_table('disaster_response_pipeline', con=engine)  
X = df['message']
y = df.drop(['id', 'message', 'genre'], axis=1)

In [83]:
# check the columns of dataset df
df.head(1)

,id,message,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [84]:
# check the columns of variable X
X.head(1)

0    Weather update - a cold front from Cuba that c...
Name: message, dtype: object

In [85]:
# check the columns of variable Y
y.head(1)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [86]:
# define tokenize function
def tokenize(text):
    """
    tokenize:   Tokenization function to process text data
    Input:      text - as string
    Returns:    clean_tokens
    """
    # tokenize text 
    tokens = word_tokenize(text)
    
    # remove stop words
    stop_words = stopwords.words("english")
    tokens = [tok for tok in tokens if tok not in stop_words]
    
    # lemmatize tokens
    lemmatizer = nltk.WordNetLemmatizer()

    # create clean tokens
    clean_tok = []
    for tok in tokens:
        clean_tok_1 = lemmatizer.lemmatize(tok).lower().strip()
        clean_tok.append(clean_tok_1)

    return clean_tok

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [87]:
# build ML pipeline
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(criterion='gini', n_estimators=50, max_depth=2, random_state=1)))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [88]:
# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y)

# train pipeline
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x143284af0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(max_depth=2,
                                                                        n_estimators=50,
                                                                        random_state=1)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [89]:
# define predict test data
y_pred = pipeline.predict(X_test)
y_test = y_test.values

In [90]:
# report f1 score precision
def ml_model(y_test, y_pred):
    """
    ml_model:   Reporting f1-score precision data.
    Input:      y_test, y_pred
    Returns:    Classification report for each column.
    """
    # get number or columns of y
    columns_count = len(y.columns)
    
    for i in range(columns_count):
        print(' ==================================================== ', '\n',
              'Report of catagory: ' + ' *** ' + y.columns[i] + ' *** ', '\n',
              '==================================================== ', '\n',
              classification_report(y_test[:, i], y_pred[:, i], zero_division=0),
              #classification_report(y_test[:, i],y_pred[:, i], output_dict=False, zero_division=1))
              #classification_report(Y_test.iloc[:,i], Y_pred.iloc[:,i])
              '==================================================== ', '\n\n')

In [91]:
# use model
ml_model(y_test, y_pred)

 Report of catagory:  *** related ***  
               precision    recall  f1-score   support

           0       0.00      0.00      0.00      1554
           1       0.76      1.00      0.87      4991

    accuracy                           0.76      6545
   macro avg       0.38      0.50      0.43      6545
weighted avg       0.58      0.76      0.66      6545


 Report of catagory:  *** request ***  
               precision    recall  f1-score   support

           0       0.83      1.00      0.91      5434
           1       0.00      0.00      0.00      1111

    accuracy                           0.83      6545
   macro avg       0.42      0.50      0.45      6545
weighted avg       0.69      0.83      0.75      6545


 Report of catagory:  *** offer ***  
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6517
           1       0.00      0.00      0.00        28

    accuracy                           1.00      6545
   ma

 Report of catagory:  *** hospitals ***  
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      6489
           1       0.00      0.00      0.00        56

    accuracy                           0.99      6545
   macro avg       0.50      0.50      0.50      6545
weighted avg       0.98      0.99      0.99      6545


 Report of catagory:  *** shops ***  
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6516
           1       0.00      0.00      0.00        29

    accuracy                           1.00      6545
   macro avg       0.50      0.50      0.50      6545
weighted avg       0.99      1.00      0.99      6545


 Report of catagory:  *** aid_centers ***  
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      6459
           1       0.00      0.00      0.00        86

    accuracy                           0.99      6545

### 6. Improve your model
Use grid search to find better parameters. 

In [96]:
# define parameters of tuned model
parameters = {
        #'vect__ngram_range': ((1, 1), (1, 2)),
        #'clf__estimator__min_samples_split': [2, 2]
        'clf__estimator__n_estimators': [100],
        'clf__estimator__max_depth': [5, 50]
    }

# use sklearn grid search to create improved model
cv = GridSearchCV(pipeline, param_grid=parameters, cv=2, verbose=3) #, n_jobs=-1)

# train cv
cv.fit(X_train, y_train)


Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] clf__estimator__max_depth=5, clf__estimator__n_estimators=100 ...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  clf__estimator__max_depth=5, clf__estimator__n_estimators=100, score=0.199, total=  22.2s
[CV] clf__estimator__max_depth=5, clf__estimator__n_estimators=100 ...


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   22.2s remaining:    0.0s


[CV]  clf__estimator__max_depth=5, clf__estimator__n_estimators=100, score=0.206, total=  59.6s
[CV] clf__estimator__max_depth=5, clf__estimator__n_estimators=100 ...


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.4min remaining:    0.0s


[CV]  clf__estimator__max_depth=5, clf__estimator__n_estimators=100, score=0.210, total=  22.2s
[CV] clf__estimator__max_depth=50, clf__estimator__n_estimators=100 ..
[CV]  clf__estimator__max_depth=50, clf__estimator__n_estimators=100, score=0.198, total= 1.2min
[CV] clf__estimator__max_depth=50, clf__estimator__n_estimators=100 ..
[CV]  clf__estimator__max_depth=50, clf__estimator__n_estimators=100, score=0.194, total= 1.2min
[CV] clf__estimator__max_depth=50, clf__estimator__n_estimators=100 ..
[CV]  clf__estimator__max_depth=50, clf__estimator__n_estimators=100, score=0.203, total= 1.2min


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:  5.2min finished


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x143284af0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(max_depth=2,
                                                                                               n_estimators=50,
                                                                                               random_state=1)))]),
             param_grid={'clf__estimator__max_depth': [5, 50],
                         'clf__estimator__n_estimators': [100]},
             verbose=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [97]:
# define predict test data of tuned model
y_pred = cv.predict(X_test)
ml_model(y_test, y_pred)

 Report of catagory:  *** related ***  
               precision    recall  f1-score   support

           0       0.00      0.00      0.00      1554
           1       0.76      1.00      0.87      4991

    accuracy                           0.76      6545
   macro avg       0.38      0.50      0.43      6545
weighted avg       0.58      0.76      0.66      6545


 Report of catagory:  *** request ***  
               precision    recall  f1-score   support

           0       0.83      1.00      0.91      5434
           1       0.00      0.00      0.00      1111

    accuracy                           0.83      6545
   macro avg       0.42      0.50      0.45      6545
weighted avg       0.69      0.83      0.75      6545


 Report of catagory:  *** offer ***  
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6517
           1       0.00      0.00      0.00        28

    accuracy                           1.00      6545
   ma

 Report of catagory:  *** shops ***  
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6516
           1       0.00      0.00      0.00        29

    accuracy                           1.00      6545
   macro avg       0.50      0.50      0.50      6545
weighted avg       0.99      1.00      0.99      6545


 Report of catagory:  *** aid_centers ***  
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      6459
           1       0.00      0.00      0.00        86

    accuracy                           0.99      6545
   macro avg       0.49      0.50      0.50      6545
weighted avg       0.97      0.99      0.98      6545


 Report of catagory:  *** other_infrastructure ***  
               precision    recall  f1-score   support

           0       0.95      1.00      0.98      6234
           1       0.00      0.00      0.00       311

    accuracy                           0.9

In [98]:
# show best parameters of ML model
cv.best_params_

{'clf__estimator__max_depth': 5, 'clf__estimator__n_estimators': 100}

In [99]:
# show best score of ML model
cv.best_score_

0.204838298955946

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [100]:
# build ML imporved ML pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('moc_knc', MultiOutputClassifier(KNeighborsClassifier()))
])

In [101]:
# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y)

# train improved pipeline
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x143284af0>)),
                ('tfidf', TfidfTransformer()),
                ('moc_knc',
                 MultiOutputClassifier(estimator=KNeighborsClassifier()))])

In [102]:
# define predict test data
y_pred = pipeline.predict(X_test)
y_test = y_test.values

In [103]:
# report f1 score precision of improved model
ml_model(y_test, y_pred)

 Report of catagory:  *** related ***  
               precision    recall  f1-score   support

           0       0.57      0.41      0.48      1584
           1       0.83      0.90      0.86      4961

    accuracy                           0.78      6545
   macro avg       0.70      0.66      0.67      6545
weighted avg       0.77      0.78      0.77      6545


 Report of catagory:  *** request ***  
               precision    recall  f1-score   support

           0       0.91      0.94      0.92      5468
           1       0.61      0.51      0.56      1077

    accuracy                           0.87      6545
   macro avg       0.76      0.72      0.74      6545
weighted avg       0.86      0.87      0.86      6545


 Report of catagory:  *** offer ***  
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      6511
           1       0.00      0.00      0.00        34

    accuracy                           0.99      6545
   ma

 Report of catagory:  *** shops ***  
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6522
           1       0.00      0.00      0.00        23

    accuracy                           1.00      6545
   macro avg       0.50      0.50      0.50      6545
weighted avg       0.99      1.00      0.99      6545


 Report of catagory:  *** aid_centers ***  
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      6471
           1       0.00      0.00      0.00        74

    accuracy                           0.99      6545
   macro avg       0.49      0.50      0.50      6545
weighted avg       0.98      0.99      0.98      6545


 Report of catagory:  *** other_infrastructure ***  
               precision    recall  f1-score   support

           0       0.95      1.00      0.97      6236
           1       0.27      0.02      0.04       309

    accuracy                           0.9

### 9. Export your model as a pickle file

In [104]:
# export model to pickle file
pickle.dump(cv,open('ml_model.pkl','wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.